<a href="https://colab.research.google.com/github/MohammadArhamKhan/.News-Article-Classification-Fake-Real/blob/main/fakeandtruenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import re #to allow special characters to be used without invoking their special meaning.
import string

In [4]:
df_fake=pd.read_csv('Fake.csv', on_bad_lines='skip')
df_true=pd.read_csv('True.csv', on_bad_lines='skip')

In [5]:
df_fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
df_fake["class"]=0
df_true["class"]=1

In [8]:
#an attribute that describes the dimensions of an array or tensor
df_fake.shape,df_true.shape

((23481, 5), (21417, 5))

In [9]:
#manual testing
df_fake_manual_testing = df_fake.tail(10)#These 10 fake news samples are set aside for manual testing (i.e., not used during training).
df_fake.drop(df_fake.tail(10).index, axis=0, inplace=True)#Removes those same 10 rows from the original df_fake dataset.Prevents data leakage — so these rows aren’t used in training.

df_true_mannual_testing = df_true.tail(10)
df_true.drop(df_true.tail(10).index, axis=0, inplace=True)

In [10]:
df_fake.shape,df_true.shape

((23471, 5), (21407, 5))

In [11]:
df_fake_manual_testing["class"]=0
df_true_mannual_testing["class"]=1

/tmp/ipython-input-11-555858607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"]=0
/tmp/ipython-input-11-555858607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_mannual_testing["class"]=1


In [12]:
df_fake_manual_testing.head()
df_true_mannual_testing.head()

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1


In [13]:
df_merge=pd.concat([df_fake,df_true],axis=0)
df_merge.head()
df_merge.columns
print(df_merge.columns)
print(df_merge.head())

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')
                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  class  
0  December 31, 2017      0  
1  December 31, 2017      0  
2  December 30, 2017      0  
3  December 29, 2017      0  
4  December 25, 2017      0  


In [14]:
df=df_merge.drop(['title','subject','date'],axis=1)
df.isnull().sum()

,0
text,0
class,0


In [15]:
#random shufling
data=df.sample(frac=1)
data.head()

,text,class
17519,MOSCOW (Reuters) - Russia said on Friday after...,1
1233,"ARLINGTON, Va. (Reuters) - The Trump administr...",1
7116,Former Speaker of the House John Boehner took ...,0
17847,MOSCOW (Reuters) - Russia accused the United S...,1
14079,HARARE (Reuters) - Zimbabwe s main opposition ...,1


In [16]:
data.reset_index(inplace=True)
data.drop(["index"],axis=1,inplace=True)
data.columns

Index(['text', 'class'], dtype='object')

In [17]:
data.head()

,text,class
0,MOSCOW (Reuters) - Russia said on Friday after...,1
1,"ARLINGTON, Va. (Reuters) - The Trump administr...",1
2,Former Speaker of the House John Boehner took ...,0
3,MOSCOW (Reuters) - Russia accused the United S...,1
4,HARARE (Reuters) - Zimbabwe s main opposition ...,1


In [18]:
#function to proceed the text
def wordopt(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub("\\W"," ",text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [19]:
data['text']=data['text'].apply(wordopt)

In [20]:
#define dependent and independent
x=data['text']
y=data['class']
#train data
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(X_train)
xv_test=vectorization.transform(X_test)

In [22]:
#logistic regresion
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
LR.fit(xv_train,y_train)
pred_lr=LR.predict(xv_test)
LR.score(xv_test,y_test)
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5770
           1       0.98      0.99      0.99      5450

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [23]:
#decission tree classification
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier()
DT.fit(xv_train,y_train)
pred_dt=DT.predict(xv_test)
DT.score(xv_test,y_test)
print(classification_report(y_test,pred_dt))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5770
           1       1.00      1.00      1.00      5450

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [24]:
#GradientBoostingclassifier
from sklearn.ensemble import GradientBoostingClassifier
GBC=GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [25]:
pred_gbc=GBC.predict(xv_test)
GBC.score(xv_test,y_test)

0.9958110516934047

In [26]:
#randomforest
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)
pred_rfc=RFC.predict(xv_test)
RFC.score(xv_test,y_test)
print(classification_report(y_test,pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5770
           1       0.99      0.99      0.99      5450

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [27]:
def output_lable(n):
  if n==0:
    return "Fake News"
  elif n==1:
    return "True News"

def manual_testing(news):
  testing_news={"text":[news]}
  new_def_test=pd.DataFrame(testing_news)
  news_def_test=pd.DataFrame(testing_news)
  new_x_test=new_def_test["text"]
  new_xv_test=vectorization.transform(new_x_test)
  pred_dt=DT.predict(new_xv_test)
  pred_lr=LR.predict(new_xv_test)
  pred_gbc=GBC.predict(new_xv_test)
  pred_rfc=RFC.predict(new_xv_test)

  return print("\n\nLR Prediction:{} \nDT prediction:{} \nRFC prediction:{}".format(output_lable(pred_lr[0]),output_lable(pred_dt[0]),output_lable(pred_rfc[0]),output_lable(pred_gbc[0])))

In [29]:
news=str(input())
manual_testing(news)

Iran’s Parliament has approved the closing of the Strait of Hormuz, though the final decision will be taken by the Supreme National Security Council, Iran’s Press TV reported on Sunday (June 22).


LR Prediction:Fake News 
DT prediction:Fake News 
RFC prediction:True News
